In [1]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity

from google.colab import drive 

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

nltk.download('stopwords')
nltk.download('punkt')

from ast import literal_eval

import math
from keras.preprocessing.text import text_to_word_sequence

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
path = 'gdrive/My Drive/MBA_USP/TCC/ifood'
df_restaurant_dish = pd.read_csv(path + '/restaurant_dish.csv',  encoding="utf-8-sig",sep=";")

In [4]:
df_restaurant_dish.shape

(5235, 10)

In [5]:
pd.options.display.max_colwidth = 500

## Tratamento dos dados

In [6]:

stop_words = stopwords.words('portuguese')
# df_restaurant_dish.drop('Unnamed: 0',inplace=True, axis=1)
df_restaurant_dish.fillna(value='',inplace=True)

def preProcessWords(text):
  words = nltk.word_tokenize(text) 
  # removendo simbolos
  words = [w.lower() for w in words if w.isalpha()] 
  # removendo stop words
  words = [w for w in words if w not in stop_words]
  # stemming
  words = [PorterStemmer().stem(w) for w in words]
  # calculando a frequência de palavras
  return " ".join(words)

df_restaurant_dish['dishDescriptionProcessed'] = df_restaurant_dish['dishDescription'].apply(preProcessWords)
df_restaurant_dish['dishDetailProcessed'] = df_restaurant_dish['dishDetails'].apply(preProcessWords)
df_restaurant_dish['paymentCodesProcessed'] = df_restaurant_dish['paymentCodes'].apply(lambda x: " ".join(literal_eval(x)))



In [7]:
df_restaurant_dish.shape

(5235, 13)

In [8]:
# Function that takes in dish descriptions as input and outputs most similar indices with scores
def get_recommendations(text, cosine_sim):
    # Get the index of the dish that matches the 
    idx = indices[text]
    if isinstance(50, list):
      idx = idx[0]

    # Get the pairwsie similarity scores of all dishes with that dish
    sim_scores = list(enumerate(cosine_sim[idx]))
    # Sort the dishes based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar dishes
    sim_scores = sim_scores[0:11]
    # Return the top 10 most similar sim_scores
    return sim_scores 

    #return df[column].iloc[dish_indices]

In [9]:
indices = pd.Series(df_restaurant_dish.index, index=df_restaurant_dish['dishDescription']).drop_duplicates()

## TF-IDF

In [10]:
# Creating TFID matrix
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df_restaurant_dish['dishDetailProcessed'])


In [11]:
tfidf_matrix.shape

(5235, 2776)

In [12]:

# Compute the cosine similarity matrix
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [13]:

def calculate_recomended_dishes(test_index, similarity_metric):
  testValue = df_restaurant_dish.loc[test_index]
  if testValue['dishDescription'] == '':
    return None
  try:
    sim_scores = get_recommendations(testValue['dishDescription'],cosine_sim=similarity_metric)
    # Get the dishes indices
    dish_indices = [i[0] for i in sim_scores]
    dish_scores =  [i[1] for i in sim_scores]
    recomendations = df_restaurant_dish['dishDescription'].loc[dish_indices]
    #print(recomendations.to_string(index=False))

    recomendation_dict = { 'Restaurant Name': df_restaurant_dish['restaurantName'].iloc[recomendations.index],
                    'Dish Description': df_restaurant_dish['dishDescription'].iloc[recomendations.index],
                    'Dish Details':  df_restaurant_dish['dishDetails'].iloc[recomendations.index],
                    'Accepted Payments':  df_restaurant_dish['paymentCodes'].iloc[recomendations.index],
                    'Similarity': dish_scores
                  }
    df_recomendation = pd.DataFrame(recomendation_dict)
    return df_recomendation
  except:
    raise Exception('Error to get similarity from ', test_index)



def calculate_all_recomended_dishes(similarity_metric, top = 10):    
  similarity_avg_list = []
  for index in df_restaurant_dish.index:
    try:
      df_recomendation = calculate_recomended_dishes(index,similarity_metric)
      if df_recomendation is None:
        continue
      similarity_avg = df_recomendation['Similarity'].iloc[1:top + 1].mean()
      if (similarity_avg == 0):
        #print('0 on index ', index)
        continue
      similarity_avg_list.append(similarity_avg)
    except Exception as e:
      continue
  print('Similariedade média com os' ,top,  'primeiros ', np.mean(similarity_avg_list))
#df_tfIdfRec.head(11).transpose()

In [14]:
#calculate_recomended_dishes(391,similarity_metric = cosine_sim)
calculate_all_recomended_dishes(similarity_metric = cosine_sim)

Similariedade média com os 10 primeiros  0.5109467919047602


## TF-IDF Multiple columns

In [15]:
df_restaurant_dish['allProcessed'] = df_restaurant_dish['dishDetailProcessed'] + df_restaurant_dish['paymentCodesProcessed'] + df_restaurant_dish['dishDescriptionProcessed']
df_restaurant_dish['allProcessed'].head()

0    sashimi salmão sashimi peix branco sushi salmão selado coberto negui shoga sushi kani sushi peix branco raspa limão sushi skin califórnia uramaki skin uramaki raspa limão uramaki crispi peix branco mini hot roll kappamaki hossomaki unidadDNR MPAY MOVPAY_MC RAM MC GPY_ELO REC RDREST GPY_MCMA AM MOVPAY_AM DIN APL_ELOD IMV MOVPAY_VIS GPY_ELOD TRE APL_MCMA GPY_MC APL_ELO MOVPAY_HIPER DNREST PIX APL_VISE VRO HIPER IFE ALR VIS RSODEX VVREST RED VIREST VR_SMA SRP VSREST APL_VIS RHIP MOVPAY_ELO GPY_...
1                                                                                                  arroz branco filé peix empanado farinha panko finalizado molho tonkatsu legum refogadoDNR MPAY MOVPAY_MC RAM MC GPY_ELO REC RDREST GPY_MCMA AM MOVPAY_AM DIN APL_ELOD IMV MOVPAY_VIS GPY_ELOD TRE APL_MCMA GPY_MC APL_ELO MOVPAY_HIPER DNREST PIX APL_VISE VRO HIPER IFE ALR VIS RSODEX VVREST RED VIREST VR_SMA SRP VSREST APL_VIS RHIP MOVPAY_ELO GPY_VISE MOVPAY_DNR ELO APL_MC GRNCAR MEREST GPY_VISmerl

In [16]:
# Creating TFID matrix
tfidf_matrix2 = tfidf.fit_transform(df_restaurant_dish['allProcessed'])

# Compute the cosine similarity matrix
cosine_sim2 = cosine_similarity(tfidf_matrix2, tfidf_matrix2)

#calculate_recomended_dishes(0,cosine_sim2)
calculate_all_recomended_dishes(similarity_metric = cosine_sim2)



Similariedade média com os 10 primeiros  0.5344280589450029


## Word Embeddings

In [17]:
path_to_glove_file = 'gdrive/My Drive/NeuralNetworkModels/glove50/glove_s50.txt'
embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs
len(embeddings_index)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  


929594

In [18]:
array_length = 20 * 50
text = df_restaurant_dish['dishDescriptionProcessed']
embedding_features = pd.DataFrame()
for document in text:
    # Saving the first 20 words of the document as a sequence
    words = text_to_word_sequence(document)[0:20] 
    
    # Retrieving the vector representation of each word and 
    # appending it to the feature vector 
    feature_vector = []
    for word in words:
        try:
            feature_vector = np.append(feature_vector, 
                                       np.array(embeddings_index[word]))
        except KeyError:
            # In the event that a word is not included in our 
            # dictionary skip that word
            pass
    # If the text has less then 20 words, fill remaining vector with
    # zeros
    zeroes_to_add = array_length - len(feature_vector)
    feature_vector = np.append(feature_vector, 
                               np.zeros(zeroes_to_add)
                               ).reshape((1,-1))
    
    # Append the document feature vector to the feature table
    embedding_features = embedding_features.append( 
                                     pd.DataFrame(feature_vector))
print(embedding_features.shape)

(5235, 1000)


In [19]:
cosine_sim3 = cosine_similarity(embedding_features, embedding_features)

#calculate_recomended_dishes(5230,cosine_sim2)
calculate_all_recomended_dishes(similarity_metric = cosine_sim3)

Similariedade média com os 10 primeiros  0.6963177396808427


In [20]:
columns = ['Restaurant Name','Dish Description','Dish Details','Similarity']
tfidf1 = calculate_recomended_dishes(391,similarity_metric = cosine_sim)
tfidf1[columns]

,Restaurant Name,Dish Description,Dish Details,Similarity
391,Delícias da Prazeres - Caxangá,Caixa Delicia,"Uma Fatia De Torta Tradicional( Bem Casado Ou Brigadeiro), 5 Doces Goumert, 12 Salgados (3 Coxinha,3 Chester,3 Bolinho De Queijo, 3 Boliviano)",1.000000
609,Portal da Carne de Sol,Bem casado,Bem casado,0.434894
1669,Padaria Jaqueira,Fatia torta de brigadeiro,Fatia de torta de brigadeiro recheada,0.365110
419,Delícias da Prazeres - Caxangá,Brownie,Recheio de chocolate ou bem casado,0.361401
392,Delícias da Prazeres - Caxangá,Mini Kit,"1mini torta 0,500kg sabor Bem casado, 20 salgados tradicionais sortidos, 10 doces tradicionais sortidos.",0.347938
767,Emporio da Picanha,15 Unidades Bolinho de Queijo,15 bolinhos de queijo,0.305496
3184,Casa dos Doces – Rosarinho,Fatia de Crocante com Doce de Leite,Fatia de torta com aproximadamente 210g,0.276063
3185,Casa dos Doces – Rosarinho,Fatia de Bem-casado,Fatia de torta com aproximadamente 210g,0.276063
3186,Casa dos Doces – Rosarinho,Fatia de Torta de Bolo de Rolo,Fatia de torta com aproximadamente 210g,0.276063
3187,Casa dos Doces – Rosarinho,Fatia de Morango com Chantilly,Fatia de torta com aproximadamente 210g,0.276063


In [21]:
tfidf2 = calculate_recomended_dishes(1,similarity_metric = cosine_sim)
tfidf2[columns]

,Restaurant Name,Dish Description,Dish Details,Similarity
1,Gendai - Rosarinho,Merluza Furai,Arroz branco com Filé de peixe empanado em farinha panko finalizado com molho tonkatsu + legumes refogados.,1.000000
4022,Kyo Sushi Delivery,Tempurá de camarão (10 und),Camarões empanados com farinha Panko.,0.457742
3861,Yantai Cozinha Oriental,Espetinho de Camarão - 3 unidades,Camarão e queijo coalho empanado na farinha panko.,0.407940
3707,Camarada Camarão - Rio Mar Recife,Praia dos carneiros,Camarões empanados com farinha panko. Acompanha molho rosé.,0.383298
2016,Seu Tito Boteco,Camarão Crocante,Filé de camarões empanados com farofa panko.,0.377964
3694,Camarada Camarão - Rio Mar Recife,Palitos de Peixe Crocante,Empanados com farinha panko e gergelim. Acompanha molho de mostarda e mel.,0.367621
3714,Camarada Camarão - Rio Mar Recife,Ao Gergelim,"Empanados com farinha panko e gergelim. Acompanha delicioso arroz refogado com tomate fresco, cebola, pimentão verde, leite de coco, queijo coalho e cebolinha e finalizado com castanha-de-caju. Acompanha geleia de pimenta.",0.366151
3741,Camarada Camarão - Rio Mar Recife,Ao Gergelim,"Empanados com farinha panko e gergelim. Acompanha delicioso arroz refogado com tomate fresco, cebola, pimentão verde, leite de coco, queijo coalho e cebolinha e finalizado com castanha-de-caju. Acompanha geleia de pimenta.",0.366151
3759,Camarada Camarão - Rio Mar Recife,Ao gergelim,"Empanados com farinha panko e gergelim. Acompanha delicioso arroz refogado com tomate fresco, cebola, pimentão verde, leite de coco, queijo coalho e cebolinha e finalizado com castanha-de-caju. Acompanha geleia de pimenta.",0.366151
31,Gendai - Rosarinho,Chicken Domburi,Arroz branco coberto com frango grelhado + legumes refogados.,0.366094


In [22]:
tfidf3 = calculate_recomended_dishes(3461,similarity_metric = cosine_sim)
tfidf3[columns]

,Restaurant Name,Dish Description,Dish Details,Similarity
3461,Eats Forneria 65,Pizza Pepperoni,"Molho especial, mussarela, pepperoni e orégano",1.000000
3512,Eats Forneria 65,Calzone pepperoni,"Molho , mussarela , pepperoni , Catupiry e orégano",0.827772
3458,Eats Forneria 65,Pizza Mussarela,"Molho especial, Mussarela, e Orégano",0.747561
3462,Eats Forneria 65,Pizza Pappermaster Reino,"Molho especial, mussarela, massa artesanal, pepperoni, queijo do reino, azeite e orégano",0.735812
3456,Eats Forneria 65,Pizza Milho,"Molho especial, mussarela, milho e orégano",0.644133
3472,Eats Forneria 65,Pizza Cheddar,"Molho especial, mussarela, cheddar e orégano",0.616835
3492,Eats Forneria 65,Pizza Bacon,"Molho especial, mussarela, bacon crocante e orégano",0.592832
3477,Eats Forneria 65,Pizza de Frango,"Molho especial, mussarela, frango temperado e orégano",0.583115
3460,Eats Forneria 65,Pizza Presunto,"Molho especial, mussarela, presunto, azeitona e orégano",0.578328
3486,Eats Forneria 65,Pizza de Camarão,"Molho especial, mussarela, camarão temperado e orégano",0.573489


In [23]:
tfidf_multiple1 = calculate_recomended_dishes(391,similarity_metric = cosine_sim2)
tfidf_multiple1[columns]

,Restaurant Name,Dish Description,Dish Details,Similarity
391,Delícias da Prazeres - Caxangá,Caixa Delicia,"Uma Fatia De Torta Tradicional( Bem Casado Ou Brigadeiro), 5 Doces Goumert, 12 Salgados (3 Coxinha,3 Chester,3 Bolinho De Queijo, 3 Boliviano)",1.000000
3189,Casa dos Doces – Rosarinho,Fatia de Bem Casado com Morango,Fatia de torta com aproximadamente 210g,0.389041
392,Delícias da Prazeres - Caxangá,Mini Kit,"1mini torta 0,500kg sabor Bem casado, 20 salgados tradicionais sortidos, 10 doces tradicionais sortidos.",0.363865
4143,Café Jaqueira,Cupcake Bem Casado,Bolinho de chocolate coberto com brigadeiro branco e chocolate,0.355468
2536,Parla Deli e Parla Pizza,Torta Bem Casado (Pequena - 15cm),,0.339543
1669,Padaria Jaqueira,Fatia torta de brigadeiro,Fatia de torta de brigadeiro recheada,0.332085
407,Delícias da Prazeres - Caxangá,Bem casado,Brigadeiro branco com brigadeiro preto,0.313439
3191,Casa dos Doces – Rosarinho,Fatia de Brigadeiro,Fatia de torta com aproximadamente 210g,0.311787
3182,Casa dos Doces – Rosarinho,Bolo no Pote de Bem Casado,,0.307836
5042,Conveniência Solar,"Sorvete Zecas Bem Casado 1,5l Exclusivos",,0.294904


In [24]:
tfidf_multiple2 = calculate_recomended_dishes(1,similarity_metric = cosine_sim2)
tfidf_multiple2[columns]

,Restaurant Name,Dish Description,Dish Details,Similarity
1,Gendai - Rosarinho,Merluza Furai,Arroz branco com Filé de peixe empanado em farinha panko finalizado com molho tonkatsu + legumes refogados.,1.000000
5,Gendai - Rosarinho,Berinjela furai,"Berinjela empanada com farinha crocante, molho tonkatsu e cebolinha - 5 unidades (Prato Vegetariano).",0.463489
31,Gendai - Rosarinho,Chicken Domburi,Arroz branco coberto com frango grelhado + legumes refogados.,0.434773
30,Gendai - Rosarinho,Beef Domburi,Arroz branco coberto com carne refogada + legumes refogados.,0.410668
1775,Pizzaria Passira - Campo Grande,Peixe à Deícia,"Filé de peixe empanado, envolvido no purê, molho branco, mussarela gratinada, banana, arroz branco e fritas",0.363541
4232,Aplânttica Pizzaria e Restaurante,1014 - Peixe ao molho de camarão,"Filé de peixe, filé de camarão, molho de coco, arroz e purê.",0.344782
1725,Pizzaria Passira - Campo Grande,Frango Grelhado,"- filé de frango na brasa, acompanha arroz branco e purê.",0.336728
1773,Pizzaria Passira - Campo Grande,Peixe ao Molho de Tomate,"Filé de Pescada amarela, c/ legumes e verduras, molho de tomate e guarnecido com arroz branco e purê",0.329708
49,Bode do Nô - Recife,Executivo - Filé de peixe tilápia,Acompanha: arroz branco e purê de batatas.,0.325419
2269,Ventura - Pizzaria e Restaurante,Executivo de Peixe à escabeche,"Filé de pescada amarela com legumes, azeitona, ervilha e molho de tomate, acompanhado de arroz branco e purê.",0.325031


In [25]:
tfidf_multiple3 = calculate_recomended_dishes(3461,similarity_metric = cosine_sim2)
tfidf_multiple3[columns]

,Restaurant Name,Dish Description,Dish Details,Similarity
3461,Eats Forneria 65,Pizza Pepperoni,"Molho especial, mussarela, pepperoni e orégano",1.000000
3512,Eats Forneria 65,Calzone pepperoni,"Molho , mussarela , pepperoni , Catupiry e orégano",0.833607
3458,Eats Forneria 65,Pizza Mussarela,"Molho especial, Mussarela, e Orégano",0.589775
3462,Eats Forneria 65,Pizza Pappermaster Reino,"Molho especial, mussarela, massa artesanal, pepperoni, queijo do reino, azeite e orégano",0.576948
3470,Eats Forneria 65,Pizza 3 Queijos,"Molho especial, mussarela, ricota, gorgonzola e orégano",0.486663
3477,Eats Forneria 65,Pizza de Frango,"Molho especial, mussarela, frango temperado e orégano",0.481973
3486,Eats Forneria 65,Pizza de Camarão,"Molho especial, mussarela, camarão temperado e orégano",0.465704
3456,Eats Forneria 65,Pizza Milho,"Molho especial, mussarela, milho e orégano",0.458085
3492,Eats Forneria 65,Pizza Bacon,"Molho especial, mussarela, bacon crocante e orégano",0.449248
3460,Eats Forneria 65,Pizza Presunto,"Molho especial, mussarela, presunto, azeitona e orégano",0.442214


In [26]:
word_embedding1 = calculate_recomended_dishes(391,similarity_metric = cosine_sim3)
word_embedding1[columns]

,Restaurant Name,Dish Description,Dish Details,Similarity
391,Delícias da Prazeres - Caxangá,Caixa Delicia,"Uma Fatia De Torta Tradicional( Bem Casado Ou Brigadeiro), 5 Doces Goumert, 12 Salgados (3 Coxinha,3 Chester,3 Bolinho De Queijo, 3 Boliviano)",1.000000
2334,Ventura - Pizzaria e Restaurante,Frango à delicia,"Filé de frango à milanesa em uma base de purê com molho e queijo mussarela gratinado, acompanhado de arroz.",0.675566
2868,Passira Pizzaria San Martin 2,Frango a delicia,"Filé de frango empanado,purê,molho branco, mussarela gratinada e arroz",0.675566
2904,Passira Pizzaria San Martin 2,Frango a delicia,"File de frango empanado, purê, molho branco, mussarela gratinada e arroz.",0.675566
5022,Conveniência Solar,Caixa Bombom Especialidades Nestle 251g,,0.668812
5021,Conveniência Solar,"Caixa Bombom Lacta Mix 250,6g",,0.492271
4546,Manu Tenório Culinária Saudável Jaqueira,Pasta amendolícia,"Feito com: amendoim, leite de amêndoas, melaço, cacau 100% e flor de sal. 200gr",0.480227
3621,Brigadissimo Lú Cloosterman Doceria,Caixa Sonha Comigo,"Caixa de 550g de gostosura base de brigadeiro chocolate gourmet, brownie de chocolate , pedaços de Sonho de Valsa e uma maravilhosa ganache de chocolate nobre. Sobremesa familia",0.477530
40,Gendai - Rosarinho,Wassabi - extra,,0.466739
4435,Granello Alimentação Saudável,Folha de louro.50g,"50g.Vitamina A e C, antioxidantes,fortalecer a imunidade por conta da vitamina C.",0.454106


In [27]:
word_embedding2 = calculate_recomended_dishes(1,similarity_metric = cosine_sim3)
word_embedding2[columns]

,Restaurant Name,Dish Description,Dish Details,Similarity
1,Gendai - Rosarinho,Merluza Furai,Arroz branco com Filé de peixe empanado em farinha panko finalizado com molho tonkatsu + legumes refogados.,1.000000
1861,Portal da Picanha,Escondidinho de Charque,Imagem meramente ilustrativa.,0.575663
2384,Ventura - Pizzaria e Restaurante,Escondidinho de charque,Charque gratinado com queijo coalho e queijo mussarela e purê de batata. Servido com arroz.,0.575663
1860,Portal da Picanha,Codorna,1 unidade\nimagem meramente ilustrativa.,0.567340
1771,Pizzaria Passira - Campo Grande,Moqueca de Peixe,"Posta de pescada amarela, legumes ao corte julienne, molho de coco, guarnecido com arroz branco e pirão .",0.534985
4119,Hasta la Pizza Espinheiro,Moqueca de peixe,"Deliciosa moqueca Original baiana com filé de tilapia, acompanha arroz branco e farofa de banana",0.534985
3711,Camarada Camarão - Rio Mar Recife,Moqueca de Banana-da-Terra,"Banana-da-terra refogada com alho, cebola, tomate, pimentões e leite de coco. Finalizada com azeite de dendê, coentro e cebolinha. Acompanham arroz com coco e farofa de dendê.",0.529278
5,Gendai - Rosarinho,Berinjela furai,"Berinjela empanada com farinha crocante, molho tonkatsu e cebolinha - 5 unidades (Prato Vegetariano).",0.518929
5137,Skillus Burger Express,Mussarela,,0.514915
2009,Seu Tito Boteco,Caldão de Caldeirada,"Caldão de frutos do mar. Acomapanha ovo de codorna, azeitonas e limão.",0.495409


In [28]:
word_embedding3 = calculate_recomended_dishes(3461,similarity_metric = cosine_sim3)
word_embedding3[columns]

,Restaurant Name,Dish Description,Dish Details,Similarity
3461,Eats Forneria 65,Pizza Pepperoni,"Molho especial, mussarela, pepperoni e orégano",1.000000
3472,Eats Forneria 65,Pizza Cheddar,"Molho especial, mussarela, cheddar e orégano",0.793758
3473,Eats Forneria 65,Pizza Gorgonzola,"Molho especial, mussarela, massa artesanal, molho pomodoro, gorgonzola e orégano",0.770140
3477,Eats Forneria 65,Pizza de Frango,"Molho especial, mussarela, frango temperado e orégano",0.749739
3479,Eats Forneria 65,Pizza Frango Crocante,"Molho especial, mussarela, frango, Catupiry maçaricado, batata palha e azeitonas",0.749739
3484,Eats Forneria 65,Pizza Catubacon Frango,"Molho especial, mussarela, frango, Catupiry e bacon crocante",0.749739
3471,Eats Forneria 65,Pizza 4 Quejos,"Molho especial, mussarela, massa artesanal, molho pomodoro, Catupiry Original, ricota, provolone e orégano",0.742779
3480,Eats Forneria 65,Pizza Frango com Catupiry,"Molho especial, mussarela, frango temperado, Catupiry e orégano",0.741156
3474,Eats Forneria 65,Pizza Gorgonberry,"Molho especial, mussarela, massa artesanal, molho pomodoro, queijo gorgonzola e geleia de pimenta",0.738113
3470,Eats Forneria 65,Pizza 3 Queijos,"Molho especial, mussarela, ricota, gorgonzola e orégano",0.736320
